In [1]:
from audio_vessel_classifier.models import model_loader
import torch


/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

def predict_data(args):

    if args["model_choice"]=="fine_tuning":
        freeze=False
    else:
        freeze= True

            
    if args["embedding_file"]:
        embedded=True
    if args["audio_file"]:

        # Configuration
        duration = 10  # in seconds
        overlap = 0.2
        desired_fs = 48000
        channel = 0
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Paths
        wav_path="/srv/DEEP-OC-underwater-noise-classification/data/G_15810_2022-04-18_23-11-14_414-573716_Cargo_underway-using-engine_11-0_4116.wav"
        # Load waveform
        waveform_info = torchaudio.info(wav_path)
        waveform, fs = torchaudio.load(wav_path)
        
        # Resample if needed
        if waveform_info.sample_rate != desired_fs:
            transform = torchaudio.transforms.Resample(fs, desired_fs)
            waveform = transform(waveform)
        
        # Crop to duration
        max_samples = int(duration * desired_fs)
        
        waveform = waveform[channel, :max_samples]
        if waveform.shape[0] < max_samples:
            waveform = F.pad(waveform, (0, max_samples - waveform.shape[0]))
        
        # Convert to numpy for processor
        x = waveform.cpu().numpy()
        
        # Initialize processor and model
        processor = ClapProcessor.from_pretrained("davidrrobinson/BioLingual")
        clap = ClapAudioModelWithProjection.from_pretrained("davidrrobinson/BioLingual")
        
        # Process inputs
        inputs = processor(audios=[x], return_tensors="pt", sampling_rate=desired_fs, padding=True)
        inputs = {key: value.to(device) for key, value in inputs.items()}
        if freeze:
            # Extract embedding
            with torch.no_grad():
                embedding = clap(**inputs).audio_embeds
        else:
            embedding= inputs["input_features"]
        print("Embedding shape:", embedding.shape)
        embedded=True
    if embedded==True:
        logger.debug("Predict with args: %s", args)
        device = return_device()
    
        try:
            update_with_query_conf(args)
            conf = config.conf_dict
    
            if "embedding_file" in args and args["embedding_file"] is not None:
                logger.debug("Using embedding file for prediction.")
                uploaded_file = args["embedding_file"]
                print(dir(uploaded_file))
                print(uploaded_file)
                embedding = torch.load(uploaded_file.filename, map_location="cpu")
    
                logger.debug("Loaded embedding: %s", type(embedding))
    
            else:
                return {"error": "No embedding file provided"}
    
        except Exception as e:
            logger.exception("Error during prediction")
            return {"error": str(e)}
    
        model = model_loader(device, freeze)
        model.eval()
        
        with torch.no_grad():
            if isinstance(embedding, torch.Tensor):
                embedding = embedding.to(device)
                if freeze:
                    output = model(embedding)[0]
                else:
                    output=model(embedding)
                predicted_class = torch.argmax(output, dim=1).item()
                probabilities = F.softmax(output, dim=1).squeeze().cpu().tolist()
        print(predicted_class, probabilities)
        return predicted_class, probabilities

In [ ]:
import soundfile as sf
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader, dataloader
import torchaudio
import torch
import numpy as np
from scipy.signal import resample
import torchvision.transforms.functional as F
import torch.nn.functional as F_general
import scipy
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import csv
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

import pathlib

from datetime import datetime

import os
import shutil


In [17]:
import torch
import torchaudio
import torch.nn.functional as F
from transformers import ClapProcessor, ClapAudioModelWithProjection

# Configuration
duration = 10  # in seconds
overlap = 0.2
desired_fs = 48000
channel = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
wav_path="/srv/DEEP-OC-underwater-noise-classification/data/G_15810_2022-04-18_23-11-14_414-573716_Cargo_underway-using-engine_11-0_4116.wav"
# Load waveform
waveform_info = torchaudio.info(wav_path)
waveform, fs = torchaudio.load(wav_path)

# Resample if needed
if waveform_info.sample_rate != desired_fs:
    transform = torchaudio.transforms.Resample(fs, desired_fs)
    waveform = transform(waveform)

# Crop to duration
max_samples = int(duration * desired_fs)

waveform = waveform[channel, :max_samples]
if waveform.shape[0] < max_samples:
    waveform = F.pad(waveform, (0, max_samples - waveform.shape[0]))

# Convert to numpy for processor
x = waveform.cpu().numpy()

# Initialize processor and model
processor = ClapProcessor.from_pretrained("davidrrobinson/BioLingual")
clap = ClapAudioModelWithProjection.from_pretrained("davidrrobinson/BioLingual")

# Process inputs
inputs = processor(audios=[x], return_tensors="pt", sampling_rate=desired_fs, padding=True)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Extract embedding
with torch.no_grad():
    embedding = clap(**inputs).audio_embeds

print("Embedding shape:", embedding.shape)


Embedding shape: torch.Size([1, 512])


In [14]:
inputs

{'input_features': tensor([[[[-13.7938, -25.0670, -30.8477,  ..., -71.3228, -72.6393, -73.0197],
           [-17.4376, -30.2243, -28.7011,  ..., -70.8276, -70.6938, -74.5322],
           [-14.8977, -17.4589, -25.4297,  ..., -72.3169, -72.0772, -74.7103],
           ...,
           [-10.7738, -22.5828, -35.8657,  ..., -69.8194, -69.4586, -70.4114],
           [-21.1377, -18.6674, -23.5634,  ..., -72.2061, -72.6141, -72.7676],
           [-17.5255, -32.5644, -27.3919,  ..., -72.6935, -71.4219, -72.6704]]]]),
 'is_longer': tensor([[False]])}

In [11]:
len(waveform)

1

In [9]:
max_samples

480000

In [ ]:
self.processor = AutoProcessor.from_pretrained("davidrrobinson/BioLingual")

In [ ]:
duration: 10  # in seconds
overlap: 0.2  # overlap of the chunks in %
desired_fs: 48000  # desired sampling rate
channel: 2  # number of channels
log: True  # whether to log or not
color: 'blue'  # color setting
 wav_path="/srv/DEEP-OC-underwater-noise-classification/data/G_15810_2022-04-18_23-11-14_414-573716_Cargo_underway-using-engine_11-0_4116.wav"

waveform_info = torchaudio.info(wav_path)

# If the selection is in between two files, open both and concatenate them
waveform, fs = torchaudio.load(wav_path) #,
                                # num_frames=461472)
# waveform, fs = torchaudio.load(wav_path,
#                                 frame_offset=row['begin_sample'],
#                                 num_frames=row['end_sample'] - row[
#                                     'begin_sample'])
if waveform_info.sample_rate != self.desired_fs:
    transform = torchaudio.transforms.Resample(fs, self.desired_fs)
    waveform = transform(waveform)
else:
    waveform = waveform

max_samples = self.max_duration * self.desired_fs
waveform = waveform[self.channel, :max_samples]
if waveform.shape[0] < max_samples:
    waveform = F_general.pad(waveform, (0, max_samples - waveform.shape[0]))
# return wav_path torch.tensor(self.label_to_id[row['label']])
x=waveform

# Ensure input data is on the correct device
x = [s.cpu().numpy() for s in x]
# Process inputs with the processor
inputs = self.processor(audios=x, return_tensors="pt", sampling_rate=self.sr, padding=True)
inputs = {key: value.to(self.device) for key, value in inputs.items()}

if self.freeze_clap:
    # Get embeddings from the CLAP model
    out = self.clap(**inputs).audio_embeds.to(self.device)
    # Save the computed embedding
    embedding=out
else:
    embedding=inputs["input_features"]

In [2]:
import torch
import io
import torch.nn.functional as F

# ----- Load the .pt file -----
pt_path = "/srv/DEEP-OC-underwater-noise-classification/data/G_15810_2022-02-07_08-21-29_156-760693_Cargo_underway-using-engine_12-3_1768 (1).pt"
with open(pt_path, "rb") as f:
    file_bytes = f.read()

file_like = io.BytesIO(file_bytes)
embedding = torch.load(file_like, map_location="cpu")

In [3]:
def return_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"Selected CUDA device: {torch.cuda.get_device_name(device)}")
    else:
        print("CUDA is not available. Using CPU.")
        device = torch.device('cpu')
    return device
device=return_device()

CUDA is not available. Using CPU.


In [4]:
model = model_loader(device, freeze=True)
model.eval()
freeze=False
with torch.no_grad():
    if isinstance(embedding, torch.Tensor):
        embedding = embedding.to(device)
        if freeze:
            output = model(embedding)[0]
        else:
            output=model(embedding)
        predicted_class = torch.argmax(output, dim=1).item()
        probabilities = F.softmax(output, dim=1).squeeze().cpu().tolist()
print(predicted_class, probabilities)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1001x64 and 512x256)

In [ ]:
from transformers import AutoProcessor, ClapModel, ClapAudioModelWithProjection, ClapProcessor
model = model_loader(device, freeze=False)
model.eval()

# ----- Run prediction -----
with torch.no_grad():
    if isinstance(embedding, torch.Tensor):
        embedding = embedding.to(device)
        output = model(embedding)
        print(output)
        predicted_class = torch.argmax(output, dim=1).item()
        probabilities = F.softmax(output, dim=1).squeeze().cpu().tolist()

        print(f"✅ Predicted class: {predicted_class}")
        print(f"✅ Class probabilities: {probabilities}")
    else:
        print("❌ Loaded object is not a tensor")

In [ ]:
def return_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"Selected CUDA device: {torch.cuda.get_device_name(device)}")
    else:
        print("CUDA is not available. Using CPU.")
        device = torch.device('cpu')
    return device
device=return_device()